In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install watchdog

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 2.0 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
model_path = '/content/drive/MyDrive/CNN_FIRE_DETECT.h5'
model = tf.keras.models.load_model(model_path)


In [ ]:
# Define the predictImage Function
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
import numpy as np

def predictImage(filename):
    img = image.load_img(filename, target_size=(150, 150))
    plt.imshow(img)

    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0  # Normalize the image

    prediction = model.predict(img_array)[0][0]  # Extract the probability score

    print(f'Prediction probability: {prediction}')

    if prediction > 0.5:
        plt.xlabel("No Fire", fontsize=30, color='red')  # Label the image
    else:
        plt.xlabel(" Fire", fontsize=30, color='green')  # Label the image

    plt.show()


In [ ]:
# Define the DriveHandler Class
import os
import time
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

class DriveHandler(FileSystemEventHandler):
    def on_created(self, event):
        if not event.is_directory:
            print(f'New file detected: {event.src_path}')
            predictImage(event.src_path)



In [ ]:
# Set Up and Start the Observer
folder_to_monitor = '/content/drive/MyDrive/ESP32-CAM/20240726/'

event_handler = DriveHandler()
observer = Observer()
observer.schedule(event_handler, path=folder_to_monitor, recursive=False)
observer.start()

try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    observer.stop()
observer.join()